In [1]:
!pip install optuna scikit-learn

In [2]:
# Load data
from transformers import AutoTokenizer, BertForSequenceClassification
from torch.nn import CrossEntropyLoss
from sklearn.metrics import classification_report, accuracy_score, f1_score
from torch.utils.data import Dataset, DataLoader
import torch
import pandas as pd
import optuna
import numpy as np
from torch.optim import AdamW


df = pd.read_excel('/kaggle/input/data-train-test/DL_train_test_data.xlsx')
# Encode severity levels
df['severity_label'] = df['severity_code']
print(df['severity_label'].unique())  # Ensure labels range from 0 to 6

# Combine short and long descriptions into a single text field
df['text'] = df['short_description'].fillna('') + ' ' + df['long_description'].fillna('')

# Tokenize data
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')
tokenized_data = tokenizer(
    df['text'].tolist(),
    padding='max_length',
    truncation=True,
    max_length=128,
    return_tensors='pt'
)

[2 6 1 4 5 0]


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [3]:

class BugDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = {k: torch.tensor(v) for k, v in encodings.items()}
        self.labels = torch.tensor(labels)

    def __getitem__(self, idx):
        return {
            'input_ids': self.encodings['input_ids'][idx],
            'attention_mask': self.encodings['attention_mask'][idx],
            'labels': self.labels[idx]
        }

    def __len__(self):
        return len(self.labels)


labels = df['severity_label'].values
dataset = BugDataset(tokenized_data, labels)

# Split dataset into training and validation sets
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=7)
model.to(device)

/tmp/ipykernel_30/1373230526.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.encodings = {k: torch.tensor(v) for k, v in encodings.items()}
You are using a model of type distilbert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'embeddings.LayerNorm.bias', 'embeddings.LayerNorm.weight', 'embeddings.position_embeddings.weight', 'embeddings.token_type_embeddings.weight', 'embeddings.word_embeddings.weight', 'encoder.layer.0.attention.output.LayerNorm.bias', 'encoder.layer.0.attention.output.LayerNorm.weight', 'encoder.layer.0.attention.output.dense.bias', 'encoder.layer.0.attention.output.dense.weight', 'encoder.layer.0.attention.self.key.bias', 'encoder.layer.0.attention.self.key.weight', 'encoder.layer.0.attention.self.query.bias', 'encoder.layer.0.attention.self.query.weight', 'encoder.layer.0.attention.self.value.bias', 'encoder.layer.0.attention.self.value.weight', 'encoder.layer.0.intermediate.dense.bias', 'encoder.layer.0.intermediate.dense.weight', 'encoder.layer.0.output.LayerNorm.bias', 'encoder.layer.0.output.LayerN

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [4]:
'''
OG Code for run 1 and 2. Only epoch, batchsize and learning rate are changed

'''

from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.utils.class_weight import compute_class_weight
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import WeightedRandomSampler
from torch.optim import AdamW
from tqdm import tqdm

# Training parameters
epochs = 10
# batch_size = best_hyperparams['batch_size']
# learning_rate = best_hyperparams['learning_rate']
batch_size = 16
learning_rate = 1e-6

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
optimizer = AdamW(model.parameters(), lr=learning_rate)
loss_fn = CrossEntropyLoss()

# Training loop
for epoch in range(epochs):
    model.train()
    train_loss = 0
    for batch in tqdm(train_loader):
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        train_loss += loss.item()
        loss.backward()
        optimizer.step()

    avg_train_loss = train_loss / len(train_loader)
    print(f"Epoch {epoch + 1}/{epochs} | Train Loss: {avg_train_loss}")

    # Validation
    model.eval()
    val_loss = 0
    preds, true_labels = [], []
    with torch.no_grad():
        for batch in tqdm(val_loader):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            val_loss += outputs.loss.item()
            preds.extend(torch.argmax(outputs.logits, dim=1).cpu().numpy())
            true_labels.extend(labels.cpu().numpy())

    # Compute metrics
    avg_val_loss = val_loss / len(val_loader)
    accuracy = accuracy_score(true_labels, preds)
    f1 = f1_score(true_labels, preds, average='weighted')  # Use 'weighted' for imbalanced classes

    print(f"Validation Loss: {avg_val_loss:.4f}")
    print(f"Validation Accuracy: {accuracy:.4f}")
    print(f"Validation F1 Score: {f1:.4f}")
    print(classification_report(true_labels, preds))


100%|██████████| 2568/2568 [15:52<00:00,  2.70it/s]


Epoch 1/10 | Train Loss: 0.6692490905836642


100%|██████████| 642/642 [01:18<00:00,  8.21it/s]
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Validation Loss: 0.6406
Validation Accuracy: 0.8397
Validation F1 Score: 0.7666
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.00      0.00      0.00       224
           2       0.84      1.00      0.91      8624
           4       0.00      0.00      0.00       677
           5       0.00      0.00      0.00       546
           6       0.00      0.00      0.00       198

    accuracy                           0.84     10270
   macro avg       0.14      0.17      0.15     10270
weighted avg       0.71      0.84      0.77     10270



100%|██████████| 2568/2568 [15:57<00:00,  2.68it/s]


...

100%|██████████| 2568/2568 [15:57<00:00,  2.68it/s]


Epoch 10/10 | Train Loss: 0.5701126709371527


100%|██████████| 642/642 [01:18<00:00,  8.15it/s]

Validation Loss: 0.5989
Validation Accuracy: 0.8420
Validation F1 Score: 0.7747
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.00      0.00      0.00       224
           2       0.84      1.00      0.91      8624
           4       0.18      0.00      0.01       677
           5       0.75      0.07      0.13       546
           6       0.00      0.00      0.00       198

    accuracy                           0.84     10270
   macro avg       0.30      0.18      0.17     10270
weighted avg       0.76      0.84      0.77     10270




/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [7]:
# Save the model
torch.save(model.state_dict(), 'optimized_model.pth')


In [ ]:
# """
# Key Changes
# Weighted Loss Function: Handles class imbalance.
# WeightedRandomSampler: Balances training samples dynamically.
# Learning Rate Scheduler: Dynamically reduces the learning rate.
# Early Stopping: Stops training if validation performance plateaus.
# Evaluation Enhancements: Logs class-wise metrics and confusion matrix

# """

# from sklearn.metrics import ConfusionMatrixDisplay
# from sklearn.utils.class_weight import compute_class_weight
# from torch.optim.lr_scheduler import StepLR
# from torch.utils.data import WeightedRandomSampler
# from torch.optim import AdamW
# from tqdm import tqdm




# # Training parameters
# epochs = 10
# batch_size = 16
# learning_rate = 7e-6

# train_labels = [labels[idx] for idx in train_dataset.indices]  # Extract labels for training dataset
# class_sample_counts = np.bincount(train_labels)
# weights = 1.0 / (class_sample_counts.astype(float) + 1)
# weights = np.nan_to_num(weights, posinf=1.0)  # Avoid inf
# sample_weights = np.array([weights[label] for label in train_labels])

# sampler = WeightedRandomSampler(sample_weights, num_samples=len(sample_weights), replacement=True)

# train_loader = DataLoader(train_dataset, batch_size=batch_size, sampler=sampler)
# val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)


# # Class weights for loss
# class_weights = torch.tensor(compute_class_weight(class_weight='balanced', classes=np.unique(labels), y=labels)).float().to(device)
# loss_fn = CrossEntropyLoss(weight=class_weights)


# # Optimizer and Scheduler
# optimizer = AdamW(model.parameters(), lr=learning_rate)
# scheduler = StepLR(optimizer, step_size=2, gamma=0.1)

# # Training loop with early stopping
# best_val_loss = float('inf')
# patience, patience_counter = 6, 0

# # Validate training and validation dataset indices
# print(f"Total dataset size: {len(dataset)}")
# print(f"Training dataset size: {len(train_dataset)}")
# print(f"Validation dataset size: {len(val_dataset)}")



# for epoch in range(epochs):
#     model.train()
#     train_loss = 0
#     count = 0
#     for batch in tqdm(train_loader):
        
#         optimizer.zero_grad()
#         input_ids = batch['input_ids'].to(device)
#         attention_mask = batch['attention_mask'].to(device)
#         labels = batch['labels'].to(device)
#         outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
#         loss = outputs.loss
#         train_loss += loss.item()
#         loss.backward()
#         optimizer.step()

#     avg_train_loss = train_loss / len(train_loader)
#     print(f"Epoch {epoch + 1}/{epochs} | Train Loss: {avg_train_loss:.4f}")

#     # Validation
#     model.eval()
#     val_loss = 0
#     preds, true_labels = [], []
#     with torch.no_grad():
#         for batch in tqdm(val_loader):
#             input_ids = batch['input_ids'].to(device)
#             attention_mask = batch['attention_mask'].to(device)
#             labels = batch['labels'].to(device)
#             outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
#             val_loss += outputs.loss.item()
#             preds.extend(torch.argmax(outputs.logits, dim=1).cpu().numpy())
#             true_labels.extend(labels.cpu().numpy())

#     # Compute metrics
#     avg_val_loss = val_loss / len(val_loader)
#     accuracy = accuracy_score(true_labels, preds)
#     f1 = f1_score(true_labels, preds, average='weighted')

#     print(f"Validation Loss: {avg_val_loss:.4f}")
#     print(f"Validation Accuracy: {accuracy:.4f}")
#     print(f"Validation F1 Score: {f1:.4f}")
#     print(classification_report(true_labels, preds))

#     # Early Stopping
#     if avg_val_loss < best_val_loss:
#         best_val_loss = avg_val_loss
#         patience_counter = 0
#         torch.save(model.state_dict(), 'best_model.pt')
#     else:
#         patience_counter += 1
#         if patience_counter >= patience:
#             print("Early stopping triggered!")
#             break

#     # Step the scheduler
#     scheduler.step()

#     # Clear GPU memory
#     torch.cuda.empty_cache()


# torch.save(model.state_dict(), 'optimized_model.pth')